In [66]:
#create google map client with api key
import gmaps
import gmaps.datasets
import googlemaps
api_key = open("../shared/shared_google_api_key.txt").readline()
gmaps.configure(api_key=api_key) # Your Google API key
google_map_api = googlemaps.Client(key=api_key)


In [ ]:
#geocode neighborhoods
import geocoder
import pandas
nyc_neighborhoods = pandas.read_csv("./street_easy_ny_neighborhoods.csv")
for index, row in nyc_neighborhoods.iterrows():
    if index % 5 == 0:
        print(str(index) + " neighborhoods geocoded" )
    current_location = geocoder.google(row["neighborhood"] + ", NY")
    if len(current_location.latlng) > 0:
        nyc_neighborhoods.set_value(index, "Lat", current_location.latlng[0])
        nyc_neighborhoods.set_value(index, "Long", current_location.latlng[1])
print("geocoding complete")

#filter neighborhoods to new york lat/long
nyc_neighborhoods = nyc_neighborhoods[
(nyc_neighborhoods["Lat"] >= 40) &
(nyc_neighborhoods["Lat"] <= 41) &
(nyc_neighborhoods["Long"] <= -73) &
(nyc_neighborhoods["Long"] >= -75)
]
#save to avoid redoing geocoding
nyc_neighborhoods.to_csv('./nyc_neighborhoods_with_location.csv')


In [17]:
#load geocoded neighborhoods
import pandas
nyc_neighborhoods = pandas.read_csv("./nyc_neighborhoods_with_location.csv")

In [63]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import re 
#crawl streat easy for average rent by neighborhood
def get_for_rent_url(location_keyword):
    location_keyword = location_keyword.replace(' ', '-')
    return 'http://streeteasy.com/for-rent/'+location_keyword+"/beds:1?page=1&sort_by=price_asc"

def get_prices(page_lines):
    prices = pd.DataFrame()
    num_prices=0 
    for i in range(0, len(page_lines)):
        #price
        current_list = page_lines[i].find_all('span',{'class':'price'})
        if len(current_list) > 0:
            price = float(current_list[0].string.replace('$','').replace(',',''))
            prices.set_value(num_prices, "price", price)
            num_prices += 1
    return prices

    
for index, row in nyc_neighborhoods.iterrows():
    #format neighborhood for url query string
    neighborhood = row["neighborhood"].replace(" ", "-")
    url=get_for_rent_url(neighborhood)
    if index % 5 == 0:
        print(str(index) + " neighborhoods processed")
    try:
        url_source=urlopen(url).read()
        decoded_source = url_source.decode('utf-8')
        soup=BeautifulSoup(url_source,'html.parser')
        potential_hit = re.search('Browse photos of (\d+)', str(url_source.decode('utf-8')))
        num_listings = potential_hit.group(1)
        page_lines=soup.find_all(lambda tag: tag.has_attr('data-id'))
        prices = get_prices(page_lines)
        average_price = prices["price"].mean()
        nyc_neighborhoods.set_value(index, "average 1BD rent", average_price)
        nyc_neighborhoods.set_value(index, "number of listings", num_listings)

    except Exception as e:
        print("error")
        print(e)
        print(neighborhood + " not found on street easy")
        nyc_neighborhoods.set_value(index, "average 1BD rent", 0)
        nyc_neighborhoods.set_value(index, "number of listings", 0)

nyc_neighborhoods.to_csv('./nyc_neighborhoods_with_locations_and_prices')

0 neighborhoods processed
error
HTTP Error 404: Not Found
Civic-Center not found on street easy
5 neighborhoods processed
error
HTTP Error 404: Not Found
Fulton/Seaport not found on street easy
10 neighborhoods processed
error
HTTP Error 404: Not Found
Lower-East-Side not found on street easy
15 neighborhoods processed
error
HTTP Error 404: Not Found
Stuyvesant-Town/PCV not found on street easy
20 neighborhoods processed
error
HTTP Error 404: Not Found
Turtle-Bay not found on street easy
25 neighborhoods processed
error
HTTP Error 404: Not Found
Hell's-Kitchen not found on street easy
30 neighborhoods processed
35 neighborhoods processed
error
HTTP Error 404: Not Found
Morningside-Heights not found on street easy
40 neighborhoods processed
error
HTTP Error 404: Not Found
Manhattanville not found on street easy
45 neighborhoods processed
error
HTTP Error 404: Not Found
Marble-Hill not found on street easy
50 neighborhoods processed
error
HTTP Error 404: Not Found
Baychester not found on

In [64]:
nyc_neighborhoods.to_csv('./nyc_neighborhoods_with_locations_and_prices')

In [ ]:

import json

json1_file = open('./all_heatmaps.json')
json1_str = json1_file.read()
all_heatmaps = json.loads(json1_str)


In [67]:
import pandas 
import googlemaps

from datetime import datetime
import time
import math
search_radius = 1609.34*.75 #meters
thai_heatmap_data = []
yoga_heatmap_data = []
starbucks_heatmap_data = []
smoke_heatmap_data = []
checks_cashed_heatmap_data = []
cent_store_heatmap_data = []
rent_heatmap_data = []
listings_heatmap_data = []
locations = []
num_locations = 0
#def add_places_nearby_to_heatmap(heatmap, location, place_keyword, search_radius):
#    places_nearby = google_map_api.places_nearby(location=current_location,keyword= place_keyword, radius=search_radius)
#    heatmap.append((row["Lat"], row["Long"], len(places_nearby["results"])))
    
for index, row in nyc_neighborhoods.iterrows():
    if num_locations % 5 == 0:
        print(str(num_locations) + " neigborhoods processed")

    num_locations += 1
    current_location = (row["Lat"], row["Long"])
    locations.append(current_location)

    nearby = google_map_api.places_nearby(location=current_location,keyword='thai restaurant', radius=search_radius)
    thai_heatmap_data.append((row["Lat"], row["Long"], len(nearby["results"])))
    nearby = google_map_api.places_nearby(location=current_location,keyword='yoga', radius=search_radius)
    yoga_heatmap_data.append((row["Lat"], row["Long"], len(nearby["results"])))
    nearby = google_map_api.places_nearby(location=current_location,keyword='starbucks', radius=search_radius)
    starbucks_heatmap_data.append((row["Lat"], row["Long"], len(nearby["results"])))
    nearby = google_map_api.places_nearby(location=current_location,keyword='smoke shop', radius=search_radius)
    smoke_heatmap_data.append((row["Lat"], row["Long"], len(nearby["results"])))
    nearby = google_map_api.places_nearby(location=current_location,keyword='checks cashed', radius=search_radius)
    checks_cashed_heatmap_data.append((row["Lat"], row["Long"], len(nearby["results"])))
    nearby = google_map_api.places_nearby(location=current_location,keyword='99 cent store', radius=search_radius)
    cent_store_heatmap_data.append((row["Lat"], row["Long"], len(nearby["results"])))
    rent_heatmap_data.append((row["Lat"], row["Long"],row["average 1BD rent"]))

    time.sleep(.02)
    

0 neigborhoods processed
5 neigborhoods processed
10 neigborhoods processed
15 neigborhoods processed
20 neigborhoods processed
25 neigborhoods processed
30 neigborhoods processed
35 neigborhoods processed
40 neigborhoods processed
45 neigborhoods processed
50 neigborhoods processed
55 neigborhoods processed
60 neigborhoods processed
65 neigborhoods processed
70 neigborhoods processed
75 neigborhoods processed
80 neigborhoods processed
85 neigborhoods processed
90 neigborhoods processed
95 neigborhoods processed
100 neigborhoods processed
105 neigborhoods processed
110 neigborhoods processed
115 neigborhoods processed
120 neigborhoods processed
125 neigborhoods processed
130 neigborhoods processed
135 neigborhoods processed
140 neigborhoods processed
145 neigborhoods processed
150 neigborhoods processed
155 neigborhoods processed
160 neigborhoods processed
165 neigborhoods processed
170 neigborhoods processed
175 neigborhoods processed
180 neigborhoods processed
185 neigborhoods proces

In [104]:


all_heatmaps = {}
all_heatmaps["thai restaurant"] = thai_heatmap_data
all_heatmaps["yoga"] = yoga_heatmap_data
all_heatmaps["checks cashed"] = checks_cashed_heatmap_data
all_heatmaps["starbucks"] = starbucks_heatmap_data
all_heatmaps["99 cent store"] = cent_store_heatmap_data
all_heatmaps["smoke"] = smoke_heatmap_data
all_heatmaps["rent"] = rent_heatmap_data
all_heatmaps["listings"] = listings_heatmap_data

for key in all_heatmaps:
    if key is not "rent":
        print(key)
        all_heatmaps[key] = normalize_by_listings(all_heatmaps[key], nyc_neighborhoods)

smoke
(40.712217200000005, -74.01605759999998, 2.75)
(40.746500399999995, -74.00137370000002, 4.5)
(40.746500399999995, -74.00137370000002, 4.5)
(40.715750899999996, -73.9970307, 3.75)
(40.714051899999994, -74.00283639999998, 4.5)
(40.7264773, -73.98153370000001, 5.0)
(40.707490899999996, -74.01127640000001, 2.0)
(40.706829299999995, -74.00438630000001, 2.25)
(40.740083, -73.9903489, 5.0)
(40.744607200000004, -73.9874105, 5.0)
(40.7191413, -73.9973273, 5.0)
(40.715033, -73.98427240000001, 2.25)
(40.7107469, -73.99696, 5.0)
(40.722897100000004, -73.99549090000001, 5.0)
(40.723301, -74.0029883, 5.0)
(40.7316903, -73.97784940000001, 3.5)
(40.716269200000006, -74.0086323, 2.5)
(40.735781, -74.0035709, 5.0)
(40.766028999999996, -73.9770582, 4.0)
(40.7549309, -73.98401949999999, 5.0)
(40.7540369, -73.9668408, 3.25)
(40.74232920000001, -73.9800645, 3.75)
(40.7478792, -73.9756567, 3.75)
(40.7694491, -73.95186199999998, 3.0)
(40.7540369, -73.9668408, 3.25)
(40.7549309, -73.98401949999999, 5.0)


In [102]:
print(all_heatmaps)

{'smoke': None, 'thai restaurant': None, 'starbucks': None, 'checks cashed': None, 'listings': None, 'rent': [(40.712217200000005, -74.01605759999998, 3.5060022977652734), (40.746500399999995, -74.00137370000002, 2.9053232961617077), (40.746500399999995, -74.00137370000002, 3.5408214395396573), (40.715750899999996, -73.9970307, 2.3152304724063497), (40.714051899999994, -74.00283639999998, 0.0), (40.7264773, -73.98153370000001, 2.6233820812513815), (40.707490899999996, -74.01127640000001, 3.4167660454695543), (40.706829299999995, -74.00438630000001, 0.0), (40.740083, -73.9903489, 3.5283522948998893), (40.744607200000004, -73.9874105, 3.987714323919409), (40.7191413, -73.9973273, 2.525836214636383), (40.715033, -73.98427240000001, 0.0), (40.7107469, -73.99696, 2.324399702000038), (40.722897100000004, -73.99549090000001, 3.2101854858736556), (40.723301, -74.0029883, 3.2422043180605904), (40.7316903, -73.97784940000001, 0.0), (40.716269200000006, -74.0086323, 4.277363737334319), (40.735781

In [69]:
import json
with open('all_heatmaps.json', 'w') as fp:
    json.dump(all_heatmaps, fp)

In [ ]:
import json

json1_file = open('./all_heatmaps.json')
json1_str = json1_file.read()
all_heatmaps = json.loads(json1_str)


In [70]:

import pandas as pd
from ipywidgets import *
from IPython.display import display
from IPython.html import widgets
from IPython.display import clear_output
import gmaps

def on_refresh_click(google_map):
    google_map = refresh_map(google_map)
    display(google_map)
    print("map refreshed")
    
def add_layer_with_properties(google_map, weighted_heatmap_data, color_gradient, layer_opacity):
    max_intensity = 0
    intensity_scale = 5
    for data_point in weighted_heatmap_data:
        if data_point[2] > max_intensity:
            max_intensity = data_point[2]
    #normalize intensities
    for i in range( 0 ,len(weighted_heatmap_data)):
        scaled_intensity = (weighted_heatmap_data[i][2]*1.00/max_intensity)*intensity_scale
        weighted_heatmap_data[i] =  (weighted_heatmap_data[i][0],weighted_heatmap_data[i][1],scaled_intensity)
    
    layer = gmaps.WeightedHeatmap(data= weighted_heatmap_data)
    layer.gradient = color_gradient
    layer.max_intensity = intensity_scale
    layer.point_radius = 35
    layer.opacity = layer_opacity
    google_map.add_layer(layer)  
    
def refresh_map(google_map):
    color_gradients = {
                        "minus": [
                                    (0, 0, 0, 0),
                                    (255, 0, 0, .7),
                                    (255, 0, 0, .8),
                                    (255, 0, 0, .99)
                                ],
                       "plus": [
                                    (0, 0, 0, 0),
                                    (0, 0, 255, .7),
                                    (0, 0, 255, .8),
                                    (0, 0, 255, .99)
                                ]
                    }
    plus_gentrification_keyword = ["thai restaurant", "yoga", "starbucks"]
    minus_gentrification_keyword = ["checks cashed", "99 cent store", "smoke"]
    num_layers = 1
    #add layers based on checkboxes
    for c in cb_container.children:
        
        if c.value:
            num_layers += 1
    #scaling for visibility with multiple layers
    opacity = .7 - num_layers*.065
    for c in cb_container.children:
        if c.value:
            if c.description in plus_gentrification_keyword:
                color_gradient = color_gradients["plus"]
            else:
                color_gradient = color_gradients["minus"]
            add_layer_with_properties(google_map, all_heatmaps[c.description], color_gradient, opacity)
    price_gradient = [(255,255,255,0),(255,255,255,.4),(255,255,255,.7),(255,255,255,.99)]
    add_layer_with_properties(google_map, all_heatmaps["rent"], price_gradient,.5)

    return google_map

checkboxes = []
cb_container = widgets.HBox()
display(cb_container)

checkboxes.append(widgets.Checkbox(description = "thai restaurant", value=True, width=90))
checkboxes.append(widgets.Checkbox(description = "yoga", value=True, width=90))
checkboxes.append(widgets.Checkbox(description = "checks cashed", value=True, width=90))
checkboxes.append(widgets.Checkbox(description = "starbucks", value=True, width=90))
checkboxes.append(widgets.Checkbox(description = "99 cent store", value=True, width=90))
checkboxes.append(widgets.Checkbox(description = "smoke", value=True, width=90))

cb_container.children=[i for i in checkboxes]


/Users/ianborukhovich/anaconda/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [72]:
#refresh map based on checkbox values
google_map = gmaps.Map()
refresh_map(google_map)
display(google_map)


In [75]:
google_map = gmaps.Map()
price_gradient = [(255,255,0,0),(255,0,0,.5),(0,0,255,.99)]
add_layer_with_properties(google_map, all_heatmaps["rent"], price_gradient,.7)
display(google_map)

In [ ]:
nyc_neighborhoods.where(nyc_neighborhoods["average 1BD rent"]>0).sort("average 1BD rent", ascending=False)